In [ ]:
import pandas
import seaborn as sns
from itertools import product
from typing import List

sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1)
sns.set_palette("colorblind")


def allocs(uppers: List[str], lowers: List[str]) -> List[str]:
    return [u + l for (u, l) in product(uppers, lowers)]

dir = "out"


# Bulk

In [ ]:
pgd = pandas.read_csv(f"{dir}/bulk_p.csv")
pgd["cores"] = pgd["x"]
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["cores", "alloc", "get", "put"]].melt(
    id_vars=["cores", "alloc"],
    value_vars=["get", "put"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable", "mem"]).min()
order = allocs(["ArrayAligned", "ArrayAtomic"], ["C64", "C128", "C256", "C512"])
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 150))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")


In [ ]:
pgd = pandas.read_csv(f"{dir}/bulk_v.csv")
pgd["cores"] = pgd["x"]
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["cores", "alloc", "get", "put"]].melt(
    id_vars=["cores", "alloc"],
    value_vars=["get", "put"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable"]).min()
order = allocs(["ArrayAligned", "ArrayAtomic"], ["C64", "C128", "C256", "C512"])
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 150))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")


# Random

In [ ]:
vd = pandas.concat([
    pandas.read_csv(f"{dir}/rand_v.csv"),
    pandas.read_csv("keep/remote/kernel_rand.csv")
], ignore_index=True)
vd["mem"] = "DRAM"
pd = data = pandas.read_csv(f"{dir}/rand_p.csv")
pd["mem"] = "NVRAM"
data = pandas.concat([vd, pd], ignore_index=True)
data["cores"] = data["x"]

# data = data.groupby(["cores", "alloc", "mem"]).min()
order = allocs(["ArrayAligned", "ArrayAtomic"], ["C64", "C128", "C256", "C512"]) + ["Kernel"]
g = sns.relplot(data=data, x="x", y="get_avg", kind="line",
                style="alloc", hue="alloc", markers=True, col="mem",
                style_order=order, hue_order=order)

g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 3000))
# g.set(xlim=(1, 17))
g.legend.set_title("Allocator")
g.set(xlabel="cores")
g.set(ylabel="time in ns")
g.set_titles("{col_name}")


# Repeat

In [ ]:
vd = pandas.concat([
    pandas.read_csv(f"{dir}/repeat_v.csv"),
    pandas.read_csv("keep/remote/kernel_repeat.csv")
], ignore_index=True)
vd["mem"] = "DRAM"
pd = pandas.read_csv(f"{dir}/repeat_p.csv")
pd["mem"] = "NVRAM"

pgd = pandas.concat([vd, pd], ignore_index=True)
pgd["cores"] = pgd["x"]
# pgd = pgd.groupby(["cores", "alloc", "mem"]).min()

order = allocs(["ArrayAligned", "ArrayAtomic"], ["P", "C64", "C256", "C512"]) + ["Kernel"]
g = sns.relplot(data=pgd, x="cores", y="get_avg", kind="line",
                style="alloc", hue="alloc", markers=True, col="mem",
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 260))
g.legend.set_title("Allocator")
g.set_titles("{col_name}")
g.set(xlabel="cores")
g.set(ylabel="time in ns")
